# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

In [ ]:
%%classpath add mvn
org.clustering4ever clustering4ever_2.11 0.9.6

In [2]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.2.1
org.apache.log4j.Logger.getRootLogger().setLevel(org.apache.log4j.Level.ERROR);

In [3]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Simple Application")
                        .master("local[4]")
                        .config("spark.ui.enabled", "false")
                        .getOrCreate()
val sc = spark.sparkContext

org.apache.spark.SparkContext@3386ca2b

[Stage 0:>                                                          (0 + 4) / 4]

# Scala _K_-Means

In [6]:
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import org.clustering4ever.clustering.patchwork.PatchWork
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal

import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import org.clustering4ever.clustering.patchwork.PatchWork
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal


## Download dataset Aggregation

In [7]:
%%bash
wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2019-08-23 11:48:39--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Résolution de www.clustering4ever.org (www.clustering4ever.org)… 62.210.16.62
Connexion à www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 8063 (7,9K) [text/csv]
Sauvegarde en : « /tmp/aggregation.csv.2 »

     0K .......                                               100%  302K=0,03s

2019-08-23 11:48:44 (302 KB/s) — « /tmp/aggregation.csv.2 » sauvegardé [8063/8063]

--2019-08-23 11:48:44--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Résolution de www.clustering4ever.org (www.clustering4ever.org)… 62.210.16.62
Connexion à www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1576 (1,5K)
Sauvegarde en : « /tmp/labels.2 »

     0K .                                                     100% 90,5M=0s



## Import data as ParArray[ArrayBuffer[Double]]

In [8]:
val datasetSize = 100000
val dim = 2
val path = "/tmp/aggregation.csv"
val useAggregationDS = true

val groundTruePath = "/tmp/labels"

val rdd = sc.parallelize(Source.fromFile(path).getLines.map( x => Array(x.split(",").map(_.toDouble):_*) ).toSeq).cache
val rddArray = rdd.map(_.toArray).cache

MapPartitionsRDD[1] at map at <console>:105

## Parameters 

In [9]:
val epsilon = Array(1.5, 1.5)
val minPts = 1
val minCellInCluster = 20
val ratio = 0.0

0.0

## Run and measure the algorithm time

In [11]:
val t1 = System.nanoTime
val model = new PatchWork(epsilon, minPts, ratio, minCellInCluster).fit(rddArray)
val t2 = System.nanoTime

(t2 - t1) / 1000000000D

6.204393079

## Different ways to apply model to datasets

In [14]:
val clusterized1 = rddArray.map( v => (model.predict(v).getID, v) ).collect

org.apache.spark.SparkException:  Task not serializable